In [ ]:
#|hide
#| eval: false
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

# Tutorial: Tabular training with Dask

> How to use bigtabular for training on large tabular datasets

To illustrate the tabular application, we will use the example of the [Adult dataset](https://archive.ics.uci.edu/ml/datasets/Adult) where we have to predict if a person is earning more or less than $50k per year using some general data.

This is a small dataset that can easily be processed in-memorory by Pandas. In practice, fast.ai's `TabularPandas` should be used when the data can be handled with Pandas. This tutorial is only to illustrate the functionality of `bigtabular` and to show the similarity to the `fastai.tabular` API. The guidance from Dask applies:

>Dask DataFrames are often used either when …
>
>  1. Your data is too big
>  2. Your computation is too slow and other techniques don’t work
>
>You should probably stick to just using pandas if …
>
>  1. Your data is small
>  2. Your computation is fast (subsecond)
>  3. There are simpler ways to accelerate your computation, like avoiding .apply or Python for loops and using a built-in pandas method instead.


In [ ]:
from fastai.tabular.all import *
from bigtabular.core import *
from bigtabular.data import *
from bigtabular.learner import *
import dask.dataframe as dd

We can download a sample of this dataset with the usual `untar_data` command:

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/home/stefan/.fastai/data/adult_sample/export.pkl'),Path('/home/stefan/.fastai/data/adult_sample/adult.csv'),Path('/home/stefan/.fastai/data/adult_sample/models')]

Then we can load the data into a Dask dataframe and have a look at how it is structured:

In [ ]:
df = pd.read_csv(path/'adult.csv')
ddf = dd.from_pandas(df)
ddf.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,<NA>,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,<NA>,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in `DaskDataLoaders` factory methods:

In [ ]:
dls = DaskDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [DaskCategorify, DaskFillMissing, DaskNormalize])

/home/stefan/Insync/OneDrive_personal/Python Workspace/bigtabular/bigtabular/core.py:193: UserWarning: `shuffle` and `drop_last` are currently ignored.
  warnings.warn('`shuffle` and `drop_last` are currently ignored.')


The last part is the list of pre-processors we apply to our data:

- `DaskCategorify` is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
- `DaskFillMissing` will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
- `DaskNormalize` will normalize the continuous variables (subtract the mean and divide by the std)

These processors are Dask compatible versions of `Categorify`, `FillMissing` and `Normalize` in `fastai.tabular`.


To further expose what's going on below the surface, let's rewrite this utilizing the `TabularDask` class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

In [ ]:
split_func = get_random_train_mask

In [ ]:
to = TabularDask(ddf, procs=[DaskCategorify, DaskFillMissing, DaskNormalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   train_mask_func=split_func)

By comparison, to show the similarity between the APIs, this is the `TabularPandas` equivalent on which `TabularDask` is based:

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [ ]:
to_ = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                    cont_names = ['age', 'fnlwgt', 'education-num'],
                    y_names='salary',
                    splits=splits)

/home/stefan/anaconda3/envs/bigtabular/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


Once we build our `TabularDask` object, our data is completely preprocessed as seen below:

In [ ]:
to.xs.head()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
0,5,8,3,0,6,5,1,0.767961,-0.831929,0.753495
1,5,13,1,5,2,5,1,0.401085,0.446923,1.535956
2,5,12,1,0,5,3,2,-0.039167,-0.880420,-0.028966
3,6,15,3,11,1,2,1,-0.039167,-0.723078,1.927187
4,7,6,3,9,6,3,2,0.254334,-1.011567,-0.028966


Now we can build our `DataLoaders` again:

In [ ]:
dls = to.dataloaders(bs=64)

/home/stefan/Insync/OneDrive_personal/Python Workspace/bigtabular/bigtabular/core.py:193: UserWarning: `shuffle` and `drop_last` are currently ignored.
  warnings.warn('`shuffle` and `drop_last` are currently ignored.')


> Later we will explore why using `TabularDask` to preprocess will be valuable.

The `show_batch` method works the same as in `fastai.tabular`:

In [ ]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Masters,Divorced,Exec-managerial,Not-in-family,White,False,44.000000,236745.999240,14.0,>=50k
1,Self-emp-not-inc,7th-8th,Married-civ-spouse,Other-service,Wife,Black,True,42.000000,82296.995365,10.0,<50k
2,Private,HS-grad,Never-married,Handlers-cleaners,Own-child,White,False,20.000000,63210.000961,9.0,<50k
3,Private,11th,Married-civ-spouse,#na#,Husband,White,False,37.000000,138940.000146,7.0,<50k
4,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,46.000000,328215.997060,9.0,>=50k
5,Private,Bachelors,Never-married,#na#,Own-child,Black,False,23.000001,529223.005361,13.0,<50k
6,Private,11th,Never-married,Adm-clerical,Own-child,White,True,18.000000,216283.999437,10.0,<50k
7,Private,Assoc-voc,Married-civ-spouse,#na#,Wife,White,True,30.000000,151989.001319,10.0,<50k
8,Private,Bachelors,Married-civ-spouse,#na#,Husband,White,True,30.000000,55290.997440,10.0,>=50k


We can define a model using the `dask_learner` method. The `DaskLearner` class inherits from the `TabularLearner` class. When we define our model, `fastai` will try to infer the loss function based on our `y_names` earlier. 

**Note**: Sometimes with tabular data, your `y`'s may be encoded (such as 0 and 1). In such a case you should explicitly pass `y_block = DaskCategoryBlock` in your constructor so `fastai` won't presume you are doing regression.

In [ ]:
learn = dask_learner(dls, metrics=accuracy)

And we can train that model with the `fit_one_cycle` method (the `fine_tune` method won't be useful here since we don't have a pretrained model).

In [ ]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.358927,0.354154,0.839822,00:51


We can then have a look at some predictions:

In [ ]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5,2,3,0,1,5,1,-0.112542,-0.476677,-1.202658,0,0
1,5,12,7,0,5,5,1,0.914712,0.896258,-0.420197,0,0
2,5,1,3,8,1,5,2,-0.039167,-0.181728,-0.028966,1,0
3,5,12,5,7,4,3,1,-0.919671,5.259980,-0.420197,0,0
4,5,16,5,13,2,3,2,0.180959,-0.461095,-0.028966,0,0
5,3,8,5,0,2,5,2,-0.699545,2.174798,-0.028966,0,0
6,7,12,3,0,1,5,2,-0.185918,0.472127,-0.028966,0,0
7,5,12,3,0,1,1,2,-0.626170,0.258673,-0.028966,0,0
8,7,10,3,0,1,5,1,1.061463,0.703579,1.144726,1,1


Or use the predict method on a row:

In [ ]:
row, clas, probs = learn.predict(ddf.head().iloc[0])

In [ ]:
row

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101319.999478,12.0


In [ ]:
clas, probs

(tensor(1), tensor([0.4177, 0.5823]))

To get prediction on a new dataframe, you can use the `test_dl` method of the `DataLoaders`. That dataframe does not need to have the dependent variable in its column.

In [ ]:
test_ddf = ddf.copy()
test_ddf = test_ddf.drop(['salary'], axis=1)
dl = learn.dls.test_dl(test_ddf)

Then `Learner.get_preds` will give you the predictions:

In [ ]:
learn.get_preds(dl=dl)

(tensor([[0.4177, 0.5823],
         [0.5308, 0.4692],
         [0.9350, 0.0650],
         ...,
         [0.5918, 0.4082],
         [0.7439, 0.2561],
         [0.7356, 0.2644]]),
 None)

:::{.callout-note}

Since machine learning models can't magically understand categories it was never trained on, the data should reflect this. If there are different missing values in your test data you should address this before training

:::